<a href="https://colab.research.google.com/github/depichan18/secure-hill-cipher-nxn/blob/main/secure_hill_cipher_nxn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix Inversion in $(n×n)$ Form as the Foundation of the Hill Cipher: Implementation and Computation Using Printable Characters


## 1. Introduction

Cryptography is the science of securing information, ensuring it can only be accessed by intended recipients. Among the classical cipher techniques, the **Hill Cipher** stands out due to its foundation in linear algebra, specifically matrix operations. Unlike monoalphabetic ciphers, the Hill Cipher applies matrix multiplication for encryption and decryption, offering better diffusion.

This project explores the implementation and computational aspects of the Hill Cipher using an **inversible square matrix of size $n \times n$**, where $n$ can range from 2 to 7. Our approach uses the **printable ASCII characters** (mod 97) to expand compatibility and usability in real-world communication systems. Moreover, the system is designed to handle both **direct user input** and **file-based encryption** using `.txt` files.

---

## 2. Theoretical Background

### 2.1 What is Hill Cipher?

The **Hill Cipher**, introduced by Lester S. Hill in 1929, is a **polygraphic substitution cipher** that uses matrix multiplication modulo a number. It encrypts blocks of text (length $n$) by treating each block as a vector and multiplying it with an $n \times n$ key matrix.

#### Encryption Formula
Let:
- $P$ be the plaintext vector,
- $K$ be the encryption matrix (size $n \times n$),
- $C$ be the ciphertext vector,
- $m$ be the modulus.

Then the encryption is:

$$
C \equiv K \cdot P \ (\text{mod } m)
$$

#### Decryption Formula
Let $K^{-1}$ be the modular inverse of the matrix $K$, then:

$$
P \equiv K^{-1} \cdot C \ (\text{mod } m)
$$

For decryption to work, $K$ must be invertible modulo $m$, which is only possible if $\det(K)$ and $m$ are coprime.

---

## 3. Innovation and Implementation

### 3.1 Innovation in This Project

This project introduces two core innovations:
- **Generalization to $n \times n$ matrices**, allowing the Hill Cipher to operate on block sizes ranging from 2 to 7. Traditional examples mostly use $2 \times 2$ or $3 \times 3$ matrices.
- **Use of printable ASCII characters** as the input alphabet, with encryption performed modulo 97. This enhances compatibility with common text formats while preserving invertibility and reducing ambiguity.

### 3.2 Why Modulo 97?

The printable ASCII characters range from ASCII code **32** (space) to **126** (tilde `~`), totaling **95 characters**. However, using **modulo 97** (a prime number) allows:
- Easier computation of matrix inverses,
- Reduced collision and ambiguity during modulo operations,
- Inclusion of padding characters like `§`, `¤` for block alignment.

### 3.3 Program Features

- **Matrix Input Size**: User can choose $n \in \{2, 3, ..., 7\}$.
- **Plaintext Input**: Text can be typed directly or uploaded via `.txt` file.
- **Automatic Padding**: Plaintext length is adjusted to be a multiple of $n$ using unused printable characters.
- **Encryption & Decryption**: Full cycle implemented with verification of accuracy.
- **Error Handling**: Ensures matrix is invertible modulo 97 before encryption proceeds.

---

## 4. Mathematical Foundation

### 4.1 Matrix Inverse Modulo $m$

To decrypt a Hill Cipher, we must compute the inverse of the matrix modulo $m$. This requires:
- Calculating $\det(K)$.
- Finding $\det(K)^{-1} \mod m$ using the **Extended Euclidean Algorithm**.
- Computing the **adjugate matrix**.
- Applying:

$$
K^{-1} \equiv \det(K)^{-1} \cdot \text{adj}(K) \mod m
$$

If $\gcd(\det(K), m) \ne 1$, the inverse does not exist, and a different key matrix must be used.

---

## 5. Case Study Example

### 5.1 Sample Encryption

- **Key Matrix Size**: $n = 3$
- **Key Matrix** (mod 97):

$$
K =
\begin{bmatrix}
2 & 4 & 5 \\
9 & 2 & 1 \\
3 & 17 & 7
\end{bmatrix}
$$

- **Plaintext**: `"hello"`
- **Padded**: `"hello¤¤"` → 6 characters (2 blocks of 3)
- **Ciphertext**: Generated in ASCII characters, readable but encrypted.

### 5.2 Decryption Process

- Compute $K^{-1} \mod 97$
- Multiply ciphertext blocks with $K^{-1}$
- Apply `mod 97` to each resulting value
- Map ASCII values back to characters

---

### 6. Computational Implementation of the Hill Cipher Using an $(n×n)$ Invertible Matrix Modulo 97

In [7]:
!pip install sympy

import sympy as sp
from google.colab import files
import io

show_help_option = input("Type 0 to view HELP or press ENTER to continue: ")
if show_help_option.strip() == '0':
    def show_help():
        print("\n=== 📘 HILL CIPHER MOD 97 PROGRAM HELP ===\n")
        print("This program encrypts and decrypts text or .txt files using the Hill Cipher algorithm.")
        print("Supported characters are printable ASCII (space to ~) plus two extra: '§' and '¤'.")
        print("Operations are performed using an nxn key matrix and modulo 97 arithmetic.\n")

        print("🔢 Main Features:")
        print("- Option to encrypt or decrypt text directly from input")
        print("- Option to encrypt or decrypt a .txt file via upload")
        print("- Automatically generates an invertible key matrix modulo 97")
        print("- Displays the key matrix and its inverse")
        print("- Provides download buttons for encryption/decryption results (for file mode)\n")

        print("✅ Usage Steps:")
        print("1. Run all cells in Google Colab")
        print("2. Enter '0' to see this help menu")
        print("3. Choose '1' for direct text or '2' for .txt file")
        print("4. Select key matrix size from 2 to 7")
        print("5. If using a .txt file, upload it when prompted")
        print("6. The encrypted and decrypted results will be shown and can be downloaded (if file mode)\n")

        print("🔐 Notes:")
        print("- Do not use characters outside ASCII 32–126 + ['§', '¤']")
        print("- Automatic padding uses '§' to complete matrix blocks")
        print("==============================================\n")

    show_help()

# CHARSET: printable ASCII (32–126) + 2 additional → total 97 characters
CHARSET = [chr(i) for i in range(32, 127)] + ['§', '¤']
CHARSET_SIZE = len(CHARSET)  # = 97
MODULUS = 97  # matches character count

# Character <-> Number mapping
char_to_num = {c: i for i, c in enumerate(CHARSET)}
num_to_char = {i: c for i, c in enumerate(CHARSET)}

def clean_text(text):
    return ''.join(c for c in text if c in CHARSET)

def pad_text(text, block_size):
    padding = (-len(text)) % block_size
    return text + '§' * padding  # padding character is guaranteed in CHARSET

def encrypt(text, key, modulus):
    size = key.shape[0]
    text = pad_text(clean_text(text), size)
    nums = [char_to_num[c] for c in text]
    blocks = [nums[i:i+size] for i in range(0, len(nums), size)]

    cipher_nums = []
    for block in blocks:
        vec = sp.Matrix(block)
        res = key * vec % modulus
        cipher_nums.extend(res)

    return ''.join(num_to_char[int(n)] for n in cipher_nums)

def decrypt(cipher, key, modulus):
    size = key.shape[0]
    nums = [char_to_num[c] for c in cipher]
    blocks = [nums[i:i+size] for i in range(0, len(nums), size)]

    inv_key = key.inv_mod(modulus)
    plain_nums = []
    for block in blocks:
        vec = sp.Matrix(block)
        res = inv_key * vec % modulus
        plain_nums.extend(res)

    return ''.join(num_to_char[int(n)] for n in plain_nums).rstrip('§')

def generate_valid_key(size, modulus):
    for _ in range(1000):
        candidate = sp.Matrix(sp.randMatrix(size, size, min=0, max=modulus - 1))
        try:
            _ = candidate.inv_mod(modulus)
            return candidate
        except:
            continue
    raise ValueError(f"Failed to find a {size}×{size} matrix that is invertible modulo {modulus}")

# === Choose Mode ===
print("Select mode:")
print("1. Encrypt/Decrypt Text")
print("2. Encrypt/Decrypt .txt File")

mode = input("Enter your choice (1/2): ").strip()
available_n = [2, 3, 4, 5, 6, 7]

while True:
    try:
        n = int(input("Choose key matrix size n x n (2–7): "))
        if n in available_n:
            break
        else:
            print("Size not available.")
    except:
        print("Please enter a valid number.")

# Generate key matrix
key = generate_valid_key(n, MODULUS)
key_inv = key.inv_mod(MODULUS)

print("\n🔐 Key Matrix:")
sp.pprint(key)

print("\n♻️ Inverse Key Matrix modulo 97:")
sp.pprint(key_inv)

# === Direct Text Mode ===
if mode == '1':
    plaintext = input("\nEnter your text message: ")
    ciphertext = encrypt(plaintext, key, MODULUS)
    decrypted = decrypt(ciphertext, key, MODULUS)

    print("\n📝 Original Message:")
    print(plaintext)

    print("\n🔒 Ciphertext:")
    print(ciphertext)

    print("\n🔓 Decrypted Text:")
    print(decrypted)

# === .txt File Mode ===
elif mode == '2':
    print("\n📤 Upload a .txt file from your device...")
    uploaded = files.upload()
    filename = next(iter(uploaded))
    text = io.StringIO(uploaded[filename].decode('utf-8')).read()
    text = clean_text(text)

    print(f"\n📄 Original file content ({filename}):\n{text}")

    ciphertext = encrypt(text, key, MODULUS)
    decrypted = decrypt(ciphertext, key, MODULUS)

    print("\n🔒 Ciphertext:")
    print(ciphertext)

    print("\n🔓 Decrypted Text:")
    print(decrypted)

    # Save results
    encrypted_filename = f"encrypted_{filename}"
    decrypted_filename = f"decrypted_{filename}"

    with open(encrypted_filename, 'w', encoding='utf-8') as f:
        f.write(ciphertext)

    with open(decrypted_filename, 'w', encoding='utf-8') as f:
        f.write(decrypted)

    print(f"\n✅ Ciphertext saved to: {encrypted_filename}")
    print(f"✅ Decrypted text saved to: {decrypted_filename}")

    files.download(encrypted_filename)
    files.download(decrypted_filename)

else:
    print("❌ Invalid mode selected.")


Type 0 to view HELP or press ENTER to continue: \
Select mode:
1. Encrypt/Decrypt Text
2. Encrypt/Decrypt .txt File
Enter your choice (1/2): 2
Choose key matrix size n x n (2–7): 7

🔐 Key Matrix:
⎡82  72  54  33  18  69  87⎤
⎢                          ⎥
⎢27  7   14  88  34  15  57⎥
⎢                          ⎥
⎢25  92  11  92  83  11  26⎥
⎢                          ⎥
⎢20  18  20  22  42  24  23⎥
⎢                          ⎥
⎢38  33  56  91  35  60  18⎥
⎢                          ⎥
⎢59  32  43  3   13  81  2 ⎥
⎢                          ⎥
⎣74  81  85  83  43  58  76⎦

♻️ Inverse Key Matrix modulo 97:
⎡45  79  84  47  60  7   68⎤
⎢                          ⎥
⎢89  36  85  48  76  44  70⎥
⎢                          ⎥
⎢12  66  61  59  75  57  23⎥
⎢                          ⎥
⎢68  96  20  22  34  7   2 ⎥
⎢                          ⎥
⎢76  37  83  77  50  55  50⎥
⎢                          ⎥
⎢68  3   34  87  5   44  20⎥
⎢                          ⎥
⎣91  67  82  86  61  31  43⎦

📤 Upload a .txt

Saving cb.txt to cb (3).txt

📄 Original file content (cb (3).txt):
hello

🔒 Ciphertext:
c.TmiNJ

🔓 Decrypted Text:
hello

✅ Ciphertext saved to: encrypted_cb (3).txt
✅ Decrypted text saved to: decrypted_cb (3).txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


## 7. Benefits and Applications

- **Educational Value**: Demonstrates real-life application of matrix theory in cryptography.
- **Flexible Security**: Larger $n$ increases security through more complex matrix transformations.
- **Compatibility**: Printable characters make the encrypted text storable and sharable.
- **Customizability**: Modular design allows adaptation to different use cases and encoding schemes.
- **File Encryption**: Useful for encrypting text-based configuration or data files securely.

---

## 8. Crucial Role of Matrix Inversion

Matrix inversion lies at the heart of the Hill Cipher decryption process. Without a valid inverse modulo $m$, it is impossible to retrieve the original plaintext from ciphertext. This makes the selection and validation of the key matrix a critical step in ensuring secure and reversible encryption.

---

## 9. Future Development

- **Padding Schemes**: Implementing standard padding like PKCS-style.
- **GUI Interface**: Making the program user-friendly.
- **Language Support**: Supporting Unicode and multilingual text beyond printable ASCII.
- **Integration with Messaging Systems**: Sending secure messages via email or chat with embedded Hill Cipher.

---

## 10. Conclusion

This project successfully demonstrates an advanced implementation of the Hill Cipher using customizable matrix sizes up to $7 \times 7$ and printable ASCII characters under modulo 97. The program emphasizes the **importance of matrix inversion** in decryption and illustrates a computational system that is both **educationally valuable** and **practically useful** for secure communication.

---


In [13]:
# Konfigurasi nama dan email kamu
!git config --global user.email "depichan18@gmail.com"
!git config --global user.name "depichan18"

# Clone ulang repo (pastikan folder lama sudah dihapus)
!git clone https://github.com/depichan18/secure-hill-cipher-nxn.git

# Copy file notebook ke dalam folder repo
!cp secure-hill-cipher-nxn.ipynb secure-hill-cipher-nxn/

# Masuk ke folder repo
%cd secure-hill-cipher-nxn

# Commit dan push
!git add .
!git commit -m "Upload from Google Colab"
!git push


Cloning into 'secure-hill-cipher-nxn'...
cp: cannot stat 'secure-hill-cipher-nxn.ipynb': No such file or directory
/content/secure-hill-cipher-nxn
On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)
error: src refspec refs/heads/main does not match any
error: failed to push some refs to 'https://github.com/depichan18/secure-hill-cipher-nxn.git'


In [12]:
# Hapus folder repo yang sudah pernah di-clone
!rm -rf secure-hill-cipher-nxn
